In [1]:
2+2

4

In [2]:
# !pip list

In [3]:
# https://partner.steamgames.com/doc/store/getreviews

In [4]:
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [5]:
# Verify that the environment variable is loaded (optional for debugging)
# print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')}")

In [6]:
import requests
import json
import csv
import sys
import os
import time  # Import time for the epoch timestamp
import re  # Import re for regex


# Construct the path to the scripts directory
script_path = os.path.abspath('../reviews-assistant/scripts')

# Add the path to sys.path
if script_path not in sys.path:
    sys.path.append(script_path)

import minsearch


class SteamReviewFetcher:
    def __init__(self, appids_with_titles, filter="all", language="english", day_range=30, review_type="all", purchase_type="all"):
        """
        Initializes the SteamReviewFetcher with the required parameters.

        :param appids_with_titles: List of tuples containing Steam application IDs and their corresponding titles.
        :param filter: Type of review filter.
        :param language: Language of the reviews.
        :param day_range: Number of days to consider for reviews.
        :param review_type: Type of review (all or specific).
        :param purchase_type: Type of purchase.
        """
        self.base_url = "https://store.steampowered.com/appreviews/"
        self.appids_with_titles = appids_with_titles if isinstance(appids_with_titles, list) else [appids_with_titles]
        self.filter = filter
        self.language = language
        self.day_range = day_range
        self.review_type = review_type
        self.purchase_type = purchase_type
        self.data_dir = os.path.abspath('../reviews-assistant/data/reviews')

        # Ensure the data directory exists
        os.makedirs(self.data_dir, exist_ok=True)

    def _construct_url(self, appid):
        return f"{self.base_url}{appid}?json=1"

    def _fetch_reviews(self, appid, num_reviews):
        url = self._construct_url(appid)
        params = {
            "filter": self.filter,
            "language": self.language,
            "day_range": self.day_range,
            "review_type": self.review_type,
            "purchase_type": self.purchase_type,
            "num_per_page": num_reviews
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()

    def get_reviews(self, num_reviews=20, print_reviews=True):
        all_reviews = {}
        for appid, title in self.appids_with_titles:
            review_data = self._fetch_reviews(appid, num_reviews)
            reviews = review_data.get("reviews", [])
            all_reviews[appid] = (title, reviews)  # Store title along with reviews
            
            if print_reviews:
                self.print_first_last_reviews(appid, title, reviews)

        return all_reviews

    def print_first_last_reviews(self, appid, title, reviews):
        total_reviews = len(reviews)
        if total_reviews == 0:
            print(f"No reviews found for App ID {appid} ({title}).")
            return

        print(f"\nFirst 5 Reviews for App ID {appid} ({title}):")
        for review in reviews[:5]:
            self._print_review(review)

        print(f"\nLast 5 Reviews for App ID {appid} ({title}):")
        for review in reviews[-5:]:
            self._print_review(review)

    def _print_review(self, review):
        print(f"Author: {review['author']['steamid']}")
        print(f"Review: {review.get('review', 'No text')}")
        print(f"Rating: {'Positive' if review['voted_up'] else 'Negative'}")
        print(f"Timestamp: {review['timestamp_created']}")
        print("-" * 79)

    def _extract_columns_to_save(self, reviews, appid, title):
        extracted_reviews = []
        current_time = int(time.time())  # Get current epoch time
        for review in reviews:
            review_dict = {
                "appid": appid,  # Move appid to the first field
                "timestamp_query": current_time,  # Move timestamp_query to the second field
                "title": title,
            }
            for column in ["recommendationid", "author.steamid", "author.playtimeforever",
                           "author.playtime_last_two_weeks", "author.playtime_at_review",
                           "author.last_played", "language", "review", "voted_up", "votes_up", "timestamp_created", "timestamp_updated"]:
                if column.split('.')[0] == 'author':
                    nested_column = column.split('.')[1]
                    review_dict[column] = review['author'].get(nested_column)
                else:
                    review_dict[column] = review.get(column)
            extracted_reviews.append(review_dict)
        return extracted_reviews

    def save_reviews(self, all_reviews, filename_prefix, format):
        for appid, (title, reviews) in all_reviews.items():
            if not reviews:
                print(f"No reviews to save for App ID {appid} ({title}).")
                continue

            safe_title = self._sanitize_title(title)  # Sanitize title for filename
            lower_safe_title = safe_title.lower()  # Convert to lowercase for the filename
            if format == "csv":
                self._save_reviews_as_csv(appid, lower_safe_title, reviews, filename_prefix, title)
            elif format == "json":
                self._save_reviews_as_json(appid, lower_safe_title, reviews, filename_prefix, title)
            else:
                print("Invalid format. Please specify 'csv' or 'json'.")

    def _sanitize_title(self, title):
        """Remove special characters from the title for safe filename."""
        return re.sub(r'[<>:"/\\|?*]', '', title).replace("'", "").replace(" ", "_")

    def _save_reviews_as_csv(self, appid, lower_safe_title, reviews, filename_prefix, title):
        keys = ['appid', 'timestamp_query', 'title', 'steamid', 'review', 'voted_up', "votes_up", 'timestamp_created']  # Update the keys
        filename = os.path.join(self.data_dir, f"{filename_prefix}_{lower_safe_title}_{appid}_reviews.csv")
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(keys)  # Write the header
            for review in reviews:
                row = [
                    appid,  # App ID as the first field
                    int(time.time()),  # Current epoch timestamp as the second field
                    title,  # Keep the original title
                    review['author']['steamid'],
                    review.get('review', 'No text'),
                    'Positive' if review['voted_up'] else 'Negative',
                    review['timestamp_created'],
                ]
                writer.writerow(row)
        print(f"Reviews for App ID {appid} ({title}) saved to {filename}")

    def _save_reviews_as_json(self, appid, lower_safe_title, reviews, filename_prefix, title):
        filename = os.path.join(self.data_dir, f"reviews_{lower_safe_title}.json")
        extracted_reviews = self._extract_columns_to_save(reviews, appid, title)  # Use original title here
        with open(filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(extracted_reviews, jsonfile, indent=4)
        print(f"Reviews for App ID {appid} ({title}) saved to {filename}")


# Example usage
if __name__ == "__main__":
    appids_with_titles = [
        ("2322010", "God of War: Ragnarok"),
        ("1086940", "Baldur's Gate 3"),
        ("1680880", "Forspoken"),
        ("1496790", "Gotham Knights"),
        ("315210", "Suicide Squad: Kill the Justice League"),
        ("2443720", "Concord"),
        ("2208920", "Assassin's Creed Valhalla"),
        ("1817070", "Marvel’s Spider-Man Remastered"),
        ("1832040", "Flintlock: The Siege of Dawn"),
        ("2698940", "The Crew Motorfest"),
        ("2702430", "Usual June"),
        ("1545560", "Shadow Gambit: The Cursed Crew"),
        ("794540", "Neo Cab"),
        ("721180", "Dustborn"),
        ("1477940", "Unknown 9: Awakening"),
        ("2239550", "Watch Dogs: Legion"),
        ("447040", "Watch_Dogs 2"),
        ("243470", "Watch_Dogs"),
        ("582160", "Assassin's Creed Origin"),
        ("812140", "Assassin's Creed Odyssey"),
        ("552520", "Far Cry 5"),
        ("2369390", "Far Cry 6"),
        ("304390", "FOR HONOR"),
        ("2842040", "Star Wars Outlaws"),
        
    ]  # List of tuples containing app IDs and their titles
    review_fetcher = SteamReviewFetcher(appids_with_titles)

    num_reviews = 10000  # Specify the number of reviews to fetch
    print_reviews_flag = False  # Set to False to turn off printing reviews

    all_reviews = review_fetcher.get_reviews(num_reviews, print_reviews=print_reviews_flag)

    # Save reviews to file
    review_fetcher.save_reviews(all_reviews, "reviews", format="json")


Reviews for App ID 2322010 (God of War: Ragnarok) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_god_of_war_ragnarok.json
Reviews for App ID 1086940 (Baldur's Gate 3) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_baldurs_gate_3.json
Reviews for App ID 1680880 (Forspoken) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_forspoken.json
Reviews for App ID 1496790 (Gotham Knights) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_gotham_knights.json
Reviews for App ID 315210 (Suicide Squad: Kill the Justice League) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_suicide_squad_kill_the_justice_league.json
Reviews for App ID 2443720 (Concord) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_concord.json
Reviews for App ID 2208920 (Assassin's Creed Valhalla) saved to /home/jovyan/reviews-assistant/data/reviews/reviews_assassins_creed_valhalla.json
Reviews for App ID 1817070 (Marvel’s Spider-Man Remastered) saved to /

# Ingestion

In [7]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list
# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561198420943538
Review: ---{ Graphics }---
✅ You forget what reality is
☐ Beautiful
☐ Good
☐ Decent
☐ Bad
☐ You will get eye cancer
☐ Get a pepper spray for your eye instead

---{ Gameplay }---
☐ Won’t ever touch any other game anymore
✅ Very good
☐ Good
☐ It's just gameplay
☐ Mehh
☐ Watch paint dry instead
☐ Tic Tac toe is better

---{ Audio }---
☐ Eargasm
✅ Very good
☐ Good
☐ Not too bad
☐ Bad
☐ I'm now deaf

---{ Audience }---
☐ Kids
✅Teens
✅ Adults
☐ Grandma

---{ PC Requirements }---
☐ Check if you can run paint
☐ Potato
☐ Decent
✅ Fast
☐ Rich boi
☐ Ask NASA if they have a spare computer
☐ Search the galaxy for dark matter fuel to run

---{ Difficulty }---
☐ Just press 'W'
☐ Easy
✅ Easy to learn / Hard to master
☐ Significant brain usage
☐ Difficult
☐ Dark Souls

---{ Grind }---
☐ Nothing to grind
☐ Only if u care about leaderboards/ranks
✅ Isn't necessary to progress
☐ Average grind level
☐ Too much grind
☐ You'll need a second life for grinding

---{ Story }---
☐ No

In [8]:
len(reviews)

1782

In [9]:
reviews[1]

{'appid': '812140',
 'timestamp_query': 1727643597,
 'title': "Assassin's Creed Odyssey",
 'recommendationid': '175243766',
 'author.steamid': '76561198835479154',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 1,
 'author.playtime_at_review': 9510,
 'author.last_played': 1726863394,
 'language': 'english',
 'review': "Despite being an ambitious game, Assassin's Creed Odyssey often feels bloated. While the vast, open-world is beautiful, it can feel repetitive and is filled with unnecessary quests that drag down the experience. The RPG mechanics, a fresh idea, sometimes take away from the Assassin's Creed experience, making it feel more like an open-world checklist simulator. That said, the story is engaging and the historical setting is captivating, especially for fans of Ancient Greece. If you have the patience for the grind, there's still fun to be had",
 'voted_up': True,
 'votes_up': 18,
 'timestamp_created': 1726669780,
 'timestamp_updated': 1726938113}

In [10]:
reviews[-1]

{'appid': '2239550',
 'timestamp_query': 1727643597,
 'title': 'Watch Dogs: Legion',
 'recommendationid': '174862159',
 'author.steamid': '76561199371555316',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 1,
 'author.playtime_at_review': 109,
 'author.last_played': 1726993288,
 'language': 'english',
 'review': 'meh',
 'voted_up': True,
 'votes_up': 0,
 'timestamp_created': 1726233314,
 'timestamp_updated': 1726233314}

In [11]:
index = minsearch.Index(
    # text_fields=["author.steamid", "author.playtimeforever", "author.playtime_last_two_weeks", "author.playtime_at_review", "author.last_played", "language", "review", "timestamp_created", "timestamp_updated"],
    text_fields=["title", "language", "review"],
    keyword_fields=["appid", "recommendationid"]
)

In [12]:
index.fit(reviews)

# RAG flow

In [13]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [14]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [15]:
prompt_template = """
You're a conservative father of young children who is not aware of how the modern gaming industry works, who is not up to date with the titles released on a daily basis.
A father who needs to keep his child from not suitable games, including DEI games.
DEI ideology, which enforces Diversity, equity, and inclusion (DEI) hurts creativity, and uses corporate propaganda to improve sales.
Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}
language: {language}
review: {review}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
prompt_template = """
You're a PC video game enthusiast who loves playing games on release day.
You're aware that it's now common for unfinished, unoptimized products to be released.
You know that developers often fail to deliver what was promised beforehand about the game.
You prefer to play games that simply work well and are delivered with proper quality—games that aren't released too early.
Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}
language: {language}
review: {review}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [19]:
question = "Is Baldur's Gate 3 a game for kids?"
answer = rag(question)
print(answer)

Baldur's Gate 3 is not specifically a game for kids. While it offers a deeply engaging and immersive experience with a rich narrative and gameplay depth that could appeal to various ages, it also contains complexities and themes that may not be suitable or comprehensible for younger audiences. The game incorporates elements from Dungeons & Dragons and features intricate mechanics that can be daunting for newcomers, which suggests it may be better suited for older players familiar with RPGs. Additionally, with player characters facing choices that can impact the story significantly, the mature themes and strategic depth indicate that it is designed more for adult gamers or those with more gaming experience.


In [20]:
question = "Is Baldur's Gate 3 a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the provided reviews of Baldur's Gate 3, it can be inferred that the game features a diversity of characters and complex narratives that appeal to a wide range of players. One review specifically mentions "the diversity of characters to play," which suggests an emphasis on character variety and representation. However, there is no explicit mention of how the game addresses equity and inclusion within its mechanics or development process.

While the game's diversity in character creation and storytelling may align with broader DEI concepts, the reviews do not provide enough details to definitively classify Baldur's Gate 3 as fully embodying DEI principles. Therefore, while the game showcases diversity in its design, specific aspects of equity and inclusion in the gameplay experience or development context remain unclear.


In [21]:
question = "Is God of War Ragnarök a game for kids?"
answer = rag(question)
print(answer)

Based on the context provided, **God of War Ragnarök** is generally not considered a game for kids. The game features mature themes, complex narratives, and intense combat, which align more with an adult audience. Additionally, there are references to violence, mythology, and emotional struggles throughout the story. While it may appeal to a wide range of players, it is more suitable for mature gamers than for a younger audience.


In [22]:
question = "Is God of War Ragnarök a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

God of War Ragnarök incorporates elements of diversity, equity, and inclusion (DEI) that some players perceive as a "woke influence." The developers collaborated with Sweet Baby Inc., a company known for inclusive storytelling, which has led to noticeable changes in plotlines and character decisions. While some players enjoy these elements, others feel they are forced and detract from the game's overall experience. This mixed reception highlights that while the game attempts to engage with DEI concepts, the effectiveness and implementation of those themes have been subjects of debate among players.


In [23]:
question = "Is Far Cry 6 a game for kids?"
answer = rag(question)
print(answer)

Based on the context provided, Far Cry 6 is not specifically labeled as a game for kids. The reviews mention elements typical of the Far Cry series, such as shooting, looting, and taking over bases, which are generally more suited for a mature audience. Additionally, it discusses themes like immersion in story and character interactions, and mentions both positive and critical aspects of the game's gameplay, indicating a level of complexity that may not appeal to younger players. Thus, it is more appropriate for an older audience rather than children.


In [24]:
question = "Is Far Cry 6 a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the reviews in the provided context, there is no indication that "Far Cry 6" explicitly follows DEI concepts, namely diversity, equity, and inclusion. The reviews focus more on gameplay, story engagement, mechanics, and comparisons to previous titles in the Far Cry series rather than discussing themes of diversity, equity, or inclusion. Therefore, we cannot conclude that "Far Cry 6" is a game following DEI concepts based on the available information.


In [25]:
question = "Is Watch Dogs: Legion a game for kids?"
answer = rag(question)
print(answer)

Based on the context provided, it does not suggest that Watch Dogs: Legion is a game specifically for kids. The reviews mention technical issues, comparisons to previous Watch Dogs games, and a remark about the depth of the story, which implies a more mature and complex gaming experience. Additionally, the content and themes typically associated with the Watch Dogs series, such as hacking and urban exploration, are not aimed at a young audience. Therefore, it can be inferred that Watch Dogs: Legion is likely more suitable for an older demographic of gamers rather than kids.


In [26]:
question = "Is Watch Dogs: Legion a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

Based on the context provided, Watch Dogs: Legion incorporates elements that align with DEI concepts, particularly through its unique "Play as Anyone" mechanic. This feature allows players to recruit and control a diverse range of characters from various backgrounds, professions, and abilities, promoting diversity within gameplay. Each character's unique skills and backstories add to the representation of different personas within the game's narrative.

However, the context does not provide specific information on how effectively these DEI concepts are implemented into the game's overarching themes or whether they resonate with players in a meaningful way. Overall, while the game features mechanics that reflect diversity and inclusion, the effectiveness and depth of these elements in relation to equity are not explicitly addressed in the reviews provided.


In [27]:
question = "Is Forspoken a game for kids?"
answer = rag(question)
print(answer)

Based on the context provided, it is not explicitly stated whether "Forspoken" is a game targeted towards kids. However, there are reviews that mention elements like a "learning curve," "campy and fun" dialogue, and a "somewhat grating" main character, which suggests that the game may appeal to a broader audience rather than being specifically for children. Additionally, the complexity of gameplay and references to mechanics such as "parkour" and "combat" imply that it may be aimed more towards older gamers. Therefore, while it may be suitable for younger players, "Forspoken" does not seem to be primarily a game for kids.


In [28]:
question = "Is Forspoken a game following DEI concepts, namely diversity, equity and inclusion?"
answer = rag(question)
print(answer)

The provided context does not explicitly discuss whether Forspoken follows DEI (diversity, equity, and inclusion) concepts. However, there is a mention of a "mostly female cast," which suggests an effort towards diversity. There is no further elaboration on equity and inclusion specifically, nor is there a comprehensive discussion of how these principles are integrated into the game's narrative or mechanics. Therefore, while some elements of diversity are noted, there isn't enough detailed information to conclusively state that Forspoken fully embraces DEI concepts in a robust manner.


In [29]:
question = "Are there any technical issues when playing Baldur's Gate 3?"
answer = rag(question)
print(answer)

Based on the reviews provided, there are no specific mentions of significant technical issues when playing Baldur's Gate 3. The overwhelming feedback highlights the game's quality, polished experience, and engaging content, suggesting that it performs well and meets players' expectations. However, one reviewer did note a concern regarding the immersion due to every character having a British accent, though this is a subjective critique rather than a technical issue. Overall, the reviews reflect a strong positive reception, indicating that the game is well-optimized and enjoyable to play.


In [30]:
question = "Is the release version of Baldur's Gate 3 ready to be played?"
answer = rag(question)
print(answer)

Yes, the release version of Baldur's Gate 3 is ready to be played. Reviews indicate it offers an amazing and polished RPG experience with a phenomenal story, engaging characters, and a well-designed combat system. Players have expressed a high level of satisfaction with the game's depth, replayability, and overall quality, noting that it feels complete and satisfying. Many also appreciate the immersive world and the seamless integration of D&D mechanics. Overall, it appears to have been well-received and is recommended for its quality gameplay experience.


In [31]:
question = "Are there any technical issues when playing God of War Ragnarok?"
answer = rag(question)
print(answer)

Yes, there are technical issues when playing God of War Ragnarok. One reviewer mentions that the game crashes approximately every two hours, which, while not frequent enough to make it unplayable, is still disappointing for a full-priced AAA game. Additionally, another review discusses issues related to frame pacing and stuttering, even though they have a powerful PC setup. The game runs well overall but can feel "off" despite a good average framerate. Furthermore, users with 4GB VRAM graphics cards encounter an error preventing the game from launching, though there is a workaround available through mods. These issues contribute to the perception that the game may not have been fully optimized for PC at launch.


In [32]:
question = "Is the release version of God of War Ragnarok ready to be played?"
answer = rag(question)
print(answer)

The release version of God of War Ragnarok on PC has several notable issues that suggest it may not be fully ready to be played for everyone. Users with 4GB VRAM graphics cards are encountering a significant error that prevents them from launching the game, as it indicates insufficient VRAM despite the game running well on weaker integrated graphics. Additionally, some players are experiencing crashes approximately every two hours, which is concerning considering the specs are above the minimum requirements. While there are some players who have reported a smooth experience, the requirement for a PSN account for a single-player game and the crashes for others further imply that the game may not be polished and optimized for all users. Thus, while some can play without issues, the overall readiness of the release version is questionable due to these bugs and limitations.


In [33]:
question = "Are there any technical issues when playing Far Cry 6?"
answer = rag(question)
print(answer)

Yes, Far Cry 6 does have technical issues. The game is reported to suffer from frequent bugs and glitches, such as disappearing enemies and missions that don't trigger properly. Additionally, players have noted long load times and inconsistent performance, including frame drops and stuttering during intense sequences. These technical problems can significantly hamper the overall gaming experience.


In [34]:
question = "Is the release version of Far Cry 6 ready to be played?"
answer = rag(question)
print(answer)

Based on the reviews provided, the release version of Far Cry 6 does not seem ready to be played due to numerous issues. One review explicitly states that the game is "so glitched one would think it's trying to rival Cyberpunk" and should not have been released in its current state. Another review warns against downloading the game on an HDD due to increased bugs and crashes. Overall, while some players find enjoyment in the gameplay and elements of the Far Cry formula, the consensus indicates significant problems with glitches and stability, implying that it may not provide a satisfactory experience upon release.
